# 공공데이터포털에서 API를 통해 개인사업자보증잔액정보를 가져옴

In [1]:
import pandas as pd
from pandas import json_normalize # json 형태의 데이터를 DataFrame으로 변환해주는 패키지
import requests # API 정보를 가져오기 위한 패키지
import json # json 형태의 데이터를 읽기 위한 패키지

In [2]:
# 공공데이터포털에 API 형태의 데이터를 요청(requests.get())
requestData = requests.get('https://apis.data.go.kr/1160100/service/GetSBBankingInfoService/getGrnBalInfo?serviceKey=DTSWD835IYOcHhgfdt0U49Lf3KjL5uUDHTfRQTCCEpwlQDihiHDdwBgjhkj7PrbBPeFweBXMcO9cK4IDvsQPYA%3D%3D&pageNo=1&numOfRows=5000&resultType=json', verify = False)
# API로 받아온 데이터의 내용을 추출해 contents에 저장
contents = requestData.text
# json 패키지의 loads()함수를 이용해 파이썬이 읽을 수 있는 자료형으로 만듦
data = json.loads(contents)
# 개인사업자재무정보.ipynb 파일 참조
stat = data['response']['body']['items']['item']
# json_normalize()함수를 통해 DataFrame으로 변환
df = json_normalize(stat)
df.columns

C:\Users\DMC CONET\miniconda3\envs\tf_gpu\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Index(['basYm', 'rprSexNm', 'rprAggrNm', 'estbYr', 'bizAreaNm', 'bizBzcCd',
       'bizBzcCdNm', 'empeCntNm', 'grnBal', 'wrkfndLoanYn', 'fctfndLoanYn'],
      dtype='object')

In [4]:
# rename()함수를 통해 영어 약어로 되어 있던 column명들을 원하는 이름으로 바꿔줌
# inplace = True 옵션을 설정하면 원래 DataFrame 내의 값을 바꿈
# inplace = False(default)일 때에는 새로운 객체를 통해 저장해줘야 함(ex. new_df = df.rename())
df.rename(columns = {'basYm' : '기준년월', 'rprSexNm' : '대표자성별명', 'rprAggrNm' : '대표자연령대명', 'estbYr' : '설립년도', 'bizAreaNm' : '사업지역명', 'bizBzcCd' : '사업업종코드',
       'bizBzcCdNm' : '사업업종명', 'empeCntNm' : '종업원수구분명', 'grnBal' : '보증잔액', 'wrkfndLoanYn' : '운전자금대출여부', 'fctfndLoanYn' : '시설자금대출여부'}, inplace = True)

In [5]:
# column명들이 모두 바뀌었음을 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   기준년월      5000 non-null   object
 1   대표자성별명    5000 non-null   object
 2   대표자연령대명   5000 non-null   object
 3   설립년도      5000 non-null   object
 4   사업지역명     5000 non-null   object
 5   사업업종코드    5000 non-null   object
 6   사업업종명     5000 non-null   object
 7   종업원수구분명   5000 non-null   object
 8   보증잔액      5000 non-null   object
 9   운전자금대출여부  5000 non-null   object
 10  시설자금대출여부  5000 non-null   object
dtypes: object(11)
memory usage: 429.8+ KB


# 다른 페이지의 정보를 가져오기 위한 코드

In [6]:
# requests.get() 내의 주소 중 pageNo 값만 변경
requestData = requests.get('https://apis.data.go.kr/1160100/service/GetSBBankingInfoService/getGrnBalInfo?serviceKey=DTSWD835IYOcHhgfdt0U49Lf3KjL5uUDHTfRQTCCEpwlQDihiHDdwBgjhkj7PrbBPeFweBXMcO9cK4IDvsQPYA%3D%3D&pageNo=10&numOfRows=5000&resultType=json', verify = False)

contents = requestData.text

data = json.loads(contents)

stat = data['response']['body']['items']['item']

df2 = json_normalize(stat)
df2.columns

C:\Users\DMC CONET\miniconda3\envs\tf_gpu\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Index(['basYm', 'rprSexNm', 'rprAggrNm', 'estbYr', 'bizAreaNm', 'bizBzcCd',
       'bizBzcCdNm', 'empeCntNm', 'grnBal', 'wrkfndLoanYn', 'fctfndLoanYn'],
      dtype='object')

In [7]:
df2.rename(columns = {'basYm' : '기준년월', 'rprSexNm' : '대표자성별명', 'rprAggrNm' : '대표자연령대명', 'estbYr' : '설립년도', 'bizAreaNm' : '사업지역명', 'bizBzcCd' : '사업업종코드',
       'bizBzcCdNm' : '사업업종명', 'empeCntNm' : '종업원수구분명', 'grnBal' : '보증잔액', 'wrkfndLoanYn' : '운전자금대출여부', 'fctfndLoanYn' : '시설자금대출여부'}, inplace = True)

# 두 페이지에서 가져온 데이터를 합쳐 csv파일로 저장

In [8]:
# 두 개의 DataFrame의 column 형태가 같으므로 pd.concat()함수를 통해 바로 합침
df3 = pd.concat([df, df2])

In [9]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   기준년월      10000 non-null  object
 1   대표자성별명    10000 non-null  object
 2   대표자연령대명   10000 non-null  object
 3   설립년도      10000 non-null  object
 4   사업지역명     10000 non-null  object
 5   사업업종코드    10000 non-null  object
 6   사업업종명     10000 non-null  object
 7   종업원수구분명   10000 non-null  object
 8   보증잔액      10000 non-null  object
 9   운전자금대출여부  10000 non-null  object
 10  시설자금대출여부  10000 non-null  object
dtypes: object(11)
memory usage: 937.5+ KB


In [10]:
# to_csv 함수를 통해 csv파일로 저장
# sep = ',' -> csv(comma seperated values)이므로 구분자는 ','
# na_rep = 'NaN' -> DataFrame의 결측값을 어떻게 저장할지 지정해주는 옵션
# encoding = 'utf-8-sig' -> csv파일을 저장할 때 한글이 있다면 'utf-8-sig'로 지정해줘야 함
df3.to_csv('C:\\Users\\DMC CONET\\Documents\\python\\project1\\data\\개인사업자보증잔액정보.csv', sep = ',', na_rep = 'NaN', encoding = 'utf-8-sig')

# 공공데이터포털에서 API를 통해 개인사업자예금대출정보를 가져옴


#### 내용은 같음

In [15]:
requestData = requests.get('https://apis.data.go.kr/1160100/service/GetSBBankingInfoService/getDpstLoanInfo?serviceKey=DTSWD835IYOcHhgfdt0U49Lf3KjL5uUDHTfRQTCCEpwlQDihiHDdwBgjhkj7PrbBPeFweBXMcO9cK4IDvsQPYA%3D%3D&pageNo=1&numOfRows=5000&resultType=json', verify = False)

contents = requestData.text

data = json.loads(contents)

stat = data['response']['body']['items']['item']

df = json_normalize(stat)
df.columns

C:\Users\DMC CONET\miniconda3\envs\tf_gpu\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Index(['basYm', 'rprSexNm', 'rprAggrNm', 'estbYr', 'bizAreaNm', 'bizBzcCd',
       'bizBzcCdNm', 'empeCntNm', 'depAbl', 'loanAbl', 'wrkfndLoanYn',
       'fctfndLoanYn'],
      dtype='object')

In [16]:
df.rename(columns = {'basYm' : '기준년월', 'rprSexNm' : '대표자성별명', 'rprAggrNm' : '대표자연령대명', 'estbYr' : '설립년도', 'bizAreaNm' : '사업지역명', 'bizBzcCd' : '사업업종코드',
       'bizBzcCdNm' : '사업업종명', 'empeCntNm' : '종업원수구분명', 'depAbl' : '예금평잔', 'loanAbl' : '대출평잔', 'wrkfndLoanYn' : '운전자금대출여부',
       'fctfndLoanYn' : '시설자금대출여부'}, inplace = True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   기준년월      5000 non-null   object
 1   대표자성별명    5000 non-null   object
 2   대표자연령대명   5000 non-null   object
 3   설립년도      5000 non-null   object
 4   사업지역명     5000 non-null   object
 5   사업업종코드    5000 non-null   object
 6   사업업종명     5000 non-null   object
 7   종업원수구분명   5000 non-null   object
 8   예금평잔      5000 non-null   object
 9   대출평잔      5000 non-null   object
 10  운전자금대출여부  5000 non-null   object
 11  시설자금대출여부  5000 non-null   object
dtypes: object(12)
memory usage: 468.9+ KB


In [18]:
requestData = requests.get('https://apis.data.go.kr/1160100/service/GetSBBankingInfoService/getDpstLoanInfo?serviceKey=DTSWD835IYOcHhgfdt0U49Lf3KjL5uUDHTfRQTCCEpwlQDihiHDdwBgjhkj7PrbBPeFweBXMcO9cK4IDvsQPYA%3D%3D&pageNo=10&numOfRows=5000&resultType=json', verify = False)

contents = requestData.text

data = json.loads(contents)

stat = data['response']['body']['items']['item']

df2 = json_normalize(stat)
df2.columns

C:\Users\DMC CONET\miniconda3\envs\tf_gpu\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Index(['basYm', 'rprSexNm', 'rprAggrNm', 'estbYr', 'bizAreaNm', 'bizBzcCd',
       'bizBzcCdNm', 'empeCntNm', 'depAbl', 'loanAbl', 'wrkfndLoanYn',
       'fctfndLoanYn'],
      dtype='object')

In [19]:
df2.rename(columns = {'basYm' : '기준년월', 'rprSexNm' : '대표자성별명', 'rprAggrNm' : '대표자연령대명', 'estbYr' : '설립년도', 'bizAreaNm' : '사업지역명', 'bizBzcCd' : '사업업종코드',
       'bizBzcCdNm' : '사업업종명', 'empeCntNm' : '종업원수구분명', 'depAbl' : '예금평잔', 'loanAbl' : '대출평잔', 'wrkfndLoanYn' : '운전자금대출여부',
       'fctfndLoanYn' : '시설자금대출여부'}, inplace = True)

In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   기준년월      5000 non-null   object
 1   대표자성별명    5000 non-null   object
 2   대표자연령대명   5000 non-null   object
 3   설립년도      5000 non-null   object
 4   사업지역명     5000 non-null   object
 5   사업업종코드    5000 non-null   object
 6   사업업종명     5000 non-null   object
 7   종업원수구분명   5000 non-null   object
 8   예금평잔      5000 non-null   object
 9   대출평잔      5000 non-null   object
 10  운전자금대출여부  5000 non-null   object
 11  시설자금대출여부  5000 non-null   object
dtypes: object(12)
memory usage: 468.9+ KB


In [21]:
df3 = pd.concat([df, df2])
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   기준년월      10000 non-null  object
 1   대표자성별명    10000 non-null  object
 2   대표자연령대명   10000 non-null  object
 3   설립년도      10000 non-null  object
 4   사업지역명     10000 non-null  object
 5   사업업종코드    10000 non-null  object
 6   사업업종명     10000 non-null  object
 7   종업원수구분명   10000 non-null  object
 8   예금평잔      10000 non-null  object
 9   대출평잔      10000 non-null  object
 10  운전자금대출여부  10000 non-null  object
 11  시설자금대출여부  10000 non-null  object
dtypes: object(12)
memory usage: 1015.6+ KB


In [22]:
df3.to_csv('C:\\Users\\DMC CONET\\Documents\\python\\project1\\data\\개인사업자예금대출정보.csv', sep = ',', na_rep = 'NaN', encoding = 'utf-8-sig')